## **Software de cálculo numérico para treliças planas**

**Introdução**

**Explicação da estrutura do software**

In [44]:
#Importando bibliotecas necessárias
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import *
import numpy as np
import xlrd

*Explicando nomes de variáveis da tabela*

<small>A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.</small>

MUDAR ISSO DEPOIS SO COPIEI E COLEI DE LA

In [45]:
#Importando as funções do arquivo funcoesTermosol.py e extraindo variáveis do arquivo de entrada
from funcoesTermosol import plota

from funcoesTermosol import importa
[nn,N,nm,Inc,nc,F,nr,R] = importa('entradaoficial.xls')

from funcoesTermosol import geraSaida

In [46]:
#Visualizando os dados obtidos através do arquivo de entrada
Inc[:,1]
Inc[:,2]
Inc
R[0,0]
Inc

array([[1.0000e+00, 2.0000e+00, 1.9314e+11, 5.2500e-06],
       [2.0000e+00, 4.0000e+00, 1.9314e+11, 5.2500e-06],
       [4.0000e+00, 6.0000e+00, 1.9314e+11, 5.2500e-06],
       [6.0000e+00, 7.0000e+00, 1.9314e+11, 5.2500e-06],
       [2.0000e+00, 3.0000e+00, 1.9314e+11, 5.2500e-06],
       [3.0000e+00, 4.0000e+00, 1.9314e+11, 5.2500e-06],
       [4.0000e+00, 5.0000e+00, 1.9314e+11, 5.2500e-06],
       [5.0000e+00, 6.0000e+00, 1.9314e+11, 5.2500e-06],
       [1.0000e+00, 3.0000e+00, 1.9314e+11, 5.2500e-06],
       [3.0000e+00, 5.0000e+00, 1.9314e+11, 5.2500e-06],
       [5.0000e+00, 7.0000e+00, 1.9314e+11, 5.2500e-06]])

In [47]:
#Montando a matriz de conectividade
m_conectividade = np.zeros((nm,7))

#preencher as incidências automaticamente
m_conectividade[:,1] = Inc[:,0]
m_conectividade[:,2] = Inc[:,1]
m_conectividade[:,4] = Inc[:,3]

for c in range(nm):
    m_conectividade[c,0] = c + 1

    #cálculo do comprimento
    x1_calculo_L = N[0,int(m_conectividade[c,1]-1)]
    y1_calculo_L = N[1,int(m_conectividade[c,1]-1)]
    x2_calculo_L = N[0,int(m_conectividade[c,2]-1)]
    y2_calculo_L = N[1,int(m_conectividade[c,2]-1)]

    m_conectividade[c,3] = sqrt((x2_calculo_L - x1_calculo_L)**2 + (y2_calculo_L - y1_calculo_L)**2)

    m_conectividade[c,6] = (y2_calculo_L - y1_calculo_L)/m_conectividade[c,3]
    m_conectividade[c,5] = (x2_calculo_L - x1_calculo_L)/m_conectividade[c,3]
    


m_conectividade

array([[ 1.00000000e+00,  1.00000000e+00,  2.00000000e+00,
         1.60996894e-01,  5.25000000e-06,  8.94427191e-01,
         4.47213595e-01],
       [ 2.00000000e+00,  2.00000000e+00,  4.00000000e+00,
         1.60996894e-01,  5.25000000e-06,  8.94427191e-01,
         4.47213595e-01],
       [ 3.00000000e+00,  4.00000000e+00,  6.00000000e+00,
         1.60996894e-01,  5.25000000e-06,  8.94427191e-01,
        -4.47213595e-01],
       [ 4.00000000e+00,  6.00000000e+00,  7.00000000e+00,
         1.60996894e-01,  5.25000000e-06,  8.94427191e-01,
        -4.47213595e-01],
       [ 5.00000000e+00,  2.00000000e+00,  3.00000000e+00,
         8.65332306e-02,  5.25000000e-06,  5.54700196e-01,
        -8.32050294e-01],
       [ 6.00000000e+00,  3.00000000e+00,  4.00000000e+00,
         1.73066461e-01,  5.25000000e-06,  5.54700196e-01,
         8.32050294e-01],
       [ 7.00000000e+00,  4.00000000e+00,  5.00000000e+00,
         1.73066461e-01,  5.25000000e-06,  5.54700196e-01,
        -8.3205029

In [48]:
#Cálculo da matriz de rigidez de cada elemento e global

m_rigidez_global = np.zeros((nn*2,nn*2))

for i in range(nm):
    multiplicador = (Inc[i,2] * Inc[i,3])/m_conectividade[i,3]
    
    s = m_conectividade[i,6]
  
    c = m_conectividade[i,5]

    m_rigidez_elemento = np.zeros((4,4))
    m_rigidez_elemento =   multiplicador * np.array([
        [c**2, c * s, -c**2, -c * s],
        [c * s, s**2, -c * s, -s**2],
        [-c**2, -c * s, c**2, c * s],
        [-c * s, -s**2, c * s, s**2]
    ])

    no_i_elemento = m_conectividade[i,1]
    no_j_elemento = m_conectividade[i,2]

    #atraves dos nós de i e j da matriz (POR IMAGEM P EXPLICAR MELHOR) calculo da posição na matriz global, a subtração vem de uma convenção
    index_i = np.array([2*no_i_elemento - 2, 2*no_i_elemento - 1], dtype=int)
    index_j = np.array([2*no_j_elemento - 2, 2*no_j_elemento - 1], dtype=int)

    #agora precisamos contemplar todas as possibilidades de combinação dos nós incidentes, então a contribuição que ele faz ao nó incidente em si (linhas 1 e 4 a seguir) e a contribuição que ele faz aos dois nós simultaneamente(linhas 2 e 3 a seguir), além disso usamos a função np.ix_ para coonseguirmos abarcar o intervalo de indices desejado 
    m_rigidez_global[np.ix_(index_i,index_i)] += m_rigidez_elemento[:2, :2]
    m_rigidez_global[np.ix_(index_i,index_j)] += m_rigidez_elemento[:2, 2:]
    m_rigidez_global[np.ix_(index_j,index_i)] += m_rigidez_elemento[2:, :2]
    m_rigidez_global[np.ix_(index_j,index_j)] += m_rigidez_elemento[2:, 2:]
    

m_rigidez_global


array([[ 1.03197038e+07,  2.51926599e+06, -5.03853197e+06,
        -2.51926599e+06, -5.28117188e+06,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 2.51926599e+06,  1.25963299e+06, -2.51926599e+06,
        -1.25963299e+06,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-5.03853197e+06, -2.51926599e+06,  1.36825619e+07,
        -3.69714935e+05, -3.60549794e+06,  5.40824691e+06,
        -5.03853197e+06, -2.51926599e+06,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-2.51926599e+06, -1.25963299e+06, -3.69714935e+05,
         1.06316363e+07,  5.40824691e+06, -8.11237036e+06,
        -2.51926599e+06, -1.25963299e+06,  0.

In [58]:
#montando a matriz de forças global e aplicando as restrições

m_rigidez_contorno = m_rigidez_global.copy()

m_rigidez_contorno[np.array(R, dtype=int), :] = 0
m_rigidez_contorno[:, np.array(R, dtype=int)] = 0
m_rigidez_contorno[np.array(R, dtype=int),np.array(R, dtype=int)] = 1

gdl_nrestritos = np.setdiff1d(np.arange(nn * 2),np.array(R, dtype = int))

m_rigidez_contorno = m_rigidez_contorno[gdl_nrestritos][:,gdl_nrestritos]
F_contorno = F[gdl_nrestritos]



array([[-1300.],
       [-1500.],
       [    0.],
       [    0.],
       [-1300.],
       [-1500.],
       [    0.],
       [    0.],
       [-1300.],
       [-1500.],
       [    0.]])

**Resolução do Sistema de Equações**

<small>A saída do sistemas de equação são as Reações de Apoio [N], Deslocamentos [m], Deformações [], Forças internas [N] e Tensões internas [Pa].</small>

In [50]:
#métodos de solução de equação - Jacobi e Gauss-Seidel

def jacobi (ite, tol, K, F):
    n = len(F)
    U = np.zeros(n)
    U_antigo = np.zeros(n)

    for k in range(ite):
        U_antigo[:] = U
        for i in range(n):
            fator_cof = np.dot(K[i, :], U_antigo) - K[i,i] * U_antigo[i]
            U[i] = (F[i] - fator_cof) / K[i,i]
        
        norma_atual = np.linalg.norm(U)
        if norma_atual > 0:
            E_i = np.linalg.norm(U - U_antigo)/norma_atual
            if E_i < tol:
                return U, E_i
        

    return U, E_i

def gauss_seidel (ite, tol, K, F):
    n = len(F)
    U = np.zeros(n)
    U_antigo = np.zeros(n)

    for k in range(ite):
        U_antigo[:] = U
        for i in range(n):
            fator_cof = np.dot(K[i, :], U) - K[i,i] * U[i]
            U[i] = (F[i] - fator_cof) / K[i,i]
        
        norma_atual = np.linalg.norm(U)
        if norma_atual > 0:
            E_i = np.linalg.norm(U - U_antigo)/norma_atual
            if E_i < tol:
                return U, E_i
        

    return U, E_i


In [77]:
#Solução com o método Jacobi

# Número máximo de iterações
max_iter = 1000
# Tolerância
tol = 1e-5

#Deslocamentos
desloc_expandido = np.zeros((nn*2,1))
desloc_expandido[gdl_nrestritos,0] = gauss_seidel(max_iter, tol,m_rigidez_contorno,F_contorno)[0]

#Reações de apoios
reacoes_apoio = np.dot(m_rigidez_global, desloc_expandido) 
reacoes_apoio = reacoes_apoio[np.array(R, dtype=int)]

reacoes_apoio

array([[[3900.4723084 ]],

       [[2899.43543001]],

       [[1599.37246476]]])

In [74]:
#Deformação
e = np.zeros((nm,1))

for i in range(nm):
   # Índices dos nós para o elemento i (ajustando para índices baseados em zero)
    no_1 = m_conectividade[i, 1] - 1
    no_2 = m_conectividade[i, 2] - 1

# Acessando os deslocamentos expandidos para os nós do elemento i
    desloc = np.array([
        desloc_expandido[int(no_1 * 2)],
        desloc_expandido[int(no_1 * 2 + 1)],
        desloc_expandido[int(no_2 * 2)],
        desloc_expandido[int(no_2 * 2 + 1)]
    ])
    
    m_def = np.array([
        [-m_conectividade[i,5], -m_conectividade[i,6], m_conectividade[i,5], m_conectividade[i,6]],
    ])
    e_elemento = (1/m_conectividade[i,3])*np.dot(m_def,desloc)
    e[i,0] = e_elemento
   
e

array([[-0.00639392],
       [-0.00449198],
       [-0.00377519],
       [-0.00352698],
       [-0.00075513],
       [ 0.00075496],
       [ 0.00191093],
       [-0.00191126],
       [ 0.00187222],
       [ 0.00103442],
       [ 0.00315463]])

In [73]:
#Deformação
sigma = np.zeros((nm,1))
tensoes = np.zeros((nm,1))

for i in range(nm):
   # Índices dos nós para o elemento i (ajustando para índices baseados em zero)
    no_1 = m_conectividade[i, 1] - 1
    no_2 = m_conectividade[i, 2] - 1

# Acessando os deslocamentos expandidos para os nós do elemento i
    desloc = np.array([
        desloc_expandido[int(no_1 * 2)],
        desloc_expandido[int(no_1 * 2 + 1)],
        desloc_expandido[int(no_2 * 2)],
        desloc_expandido[int(no_2 * 2 + 1)]
    ])
    m_def = np.array([
        [-m_conectividade[i,5], -m_conectividade[i,6], m_conectividade[i,5], m_conectividade[i,6]],
    ])
    sigma_elemento = (Inc[i,2]/m_conectividade[i,3])*np.dot(m_def,desloc)
    sigma[i,0] = sigma_elemento
    tensoes[i,0] = sigma_elemento*m_conectividade[i,4]
    
sigma

array([[-1.23492090e+09],
       [-8.67581536e+08],
       [-7.29140166e+08],
       [-6.81201058e+08],
       [-1.45846200e+08],
       [ 1.45812269e+08],
       [ 3.69077504e+08],
       [-3.69141002e+08],
       [ 3.61599724e+08],
       [ 1.99788026e+08],
       [ 6.09284748e+08]])

In [ ]:
geraSaida('saida',reacoes_apoio,desloc_expandido,e,tensoes,sigma)